In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
import os

# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.compat.v1.Session(config=config)

In [2]:
current_dir = os.getcwd()

In [3]:
current_dir

'C:\\Users\\wavac\\Documents\\GitHub\\bangkit-final-JKT2-E'

In [4]:
TRAINING_DIR = current_dir+"\\dataset\\asl_alphabet_train\\asl_alphabet_train\\"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.2)

# training_datagen = ImageDataGenerator(
#       rescale = 1./255,
#       horizontal_flip=True,
#       fill_mode='nearest',
#       validation_split=0.2)

# VALIDATION_DIR = current_dir+"\\dataset\\asl_alphabet_test\\asl_alphabet_test\\"
# validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=128,
    subset='training'
)

validation_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126,
    subset='validation'
)

classes = 29

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


### Pre-trained : MobileNet

In [20]:
#MobileNet

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(classes,activation='softmax')(x) #final layer with softmax activation
mobile_model=Model(inputs=base_model.input,outputs=preds)

from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor = 'val_accuracy',
                          mode = 'max',
                        patience=1,
                        min_delta = 0,
                        restore_best_weights = True)

for layer in mobile_model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in mobile_model.layers[:20]:
    layer.trainable=False
for layer in mobile_model.layers[20:]:
    layer.trainable=True

In [21]:
mobile_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [22]:
mobile_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train= train_generator.n//train_generator.batch_size

MODEL_FILE = 'mobile_model.h5'
mobile_model.fit(train_generator, epochs=5, steps_per_epoch=step_size_train, callbacks=[earlystop], validation_data = validation_generator, verbose = 1, validation_steps=3)
mobile_model.save(MODEL_FILE)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 543 steps, validate for 3 steps
Epoch 1/5
543/543 [==============================] - 401s 739ms/step - loss: 0.4824 - accuracy: 0.8721 - val_loss: 7.2564 - val_accuracy: 0.3280
Epoch 2/5
543/543 [==============================] - 422s 778ms/step - loss: 0.2001 - accuracy: 0.9535 - val_loss: 1.8381 - val_accuracy: 0.5423
Epoch 3/5
543/543 [==============================] - 431s 793ms/step - loss: 0.1510 - accuracy: 0.9648 - val_loss: 2.0807 - val_accuracy: 0.4603


### Pre-trained : Inception

In [47]:
#Inception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

classes = 29
base_model = InceptionV3(weights='imagenet', include_top=False)
out = base_model.output
out = GlobalAveragePooling2D(name='avg_pool')(out)
out = Dropout(0.4)(out)
predictions = Dense(classes, activation='softmax')(out)
inception_model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
inception_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_192 (Conv2D)             (None, None, None, 3 864         input_11[0][0]                   
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, None, None, 3 96          conv2d_192[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, None, None, 3 0           batch_normalization_188[0][0]    
____________________________________________________________________________________________

In [48]:
inception_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
EPOCHS = 25
MODEL_FILE = 'inception_model.h5'
VALIDATION_STEPS = 3
STEPS_PER_EPOCH = 271

history = inception_model.fit(train_generator, epochs=25, steps_per_epoch=543, validation_data = validation_generator, verbose = 1, validation_steps=3)
inception_model.save(MODEL_FILE)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 543 steps, validate for 3 steps
Epoch 1/25
543/543 [==============================] - 135s 249ms/step - loss: 1.3754 - accuracy: 0.6439 - val_loss: 6.2855 - val_accuracy: 0.1455
Epoch 2/25
543/543 [==============================] - 84s 155ms/step - loss: 0.7399 - accuracy: 0.7993 - val_loss: 7.2854 - val_accuracy: 0.1799
Epoch 3/25
543/543 [==============================] - 87s 160ms/step - loss: 0.6330 - accuracy: 0.8217 - val_loss: 8.4883 - val_accuracy: 0.1614
Epoch 4/25
543/543 [==============================] - 83s 154ms/step - loss: 0.5879 - accuracy: 0.8304 - val_loss: 8.8268 - val_accuracy: 0.1799
Epoch 5/25
543/543 [==============================] - 84s 154ms/step - loss: 0.5522 - accuracy: 0.8393 - val_loss: 8.5699 - val_accuracy: 0.1958
Epoch 6/25
543/543 [==============================] - 83s 153ms/step - loss: 0.5386 - accuracy: 0.8422 - val_loss: 8.8056 - val_accuracy: 0.1958
Epoch 7/25
543/543 [=================

### Pre-trained : VGG

In [17]:
#VGG Model

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.layers import Dense, Activation, Flatten

vgg_base = VGG16(include_top=False, input_shape=(150,150,3),weights='imagenet')
vgg_base.trainable = False

vgg_model = keras.models.Sequential()
vgg_model.add(vgg_base)
vgg_model.add(Flatten())
vgg_model.add(Dense(29, activation='softmax'))

vgg_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 29)                237597    
Total params: 14,952,285
Trainable params: 237,597
Non-trainable params: 14,714,688
_________________________________________________________________


In [19]:
vgg_model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

history = vgg_model.fit(train_generator, epochs=25, steps_per_epoch=271, validation_data = validation_generator, verbose = 1, validation_steps=3)

vgg_model.save(current_dir+"\\vgg_model.h5")

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 271 steps, validate for 3 steps
Epoch 1/25
  1/271 [..............................] - ETA: 5:46

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_3/vgg16/block1_conv1/Conv2D (defined at <ipython-input-19-6ee3dbe75d63>:3) ]] [Op:__inference_distributed_function_6321]

Function call stack:
distributed_function


### Baseline/Benchmark Model

In [28]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
#     tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(29, activation='softmax')
])


model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 82944)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 82944)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)              

In [29]:
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


step_size_train= train_generator.n//train_generator.batch_size
history = model.fit(train_generator, epochs=5, steps_per_epoch=step_size_train, validation_data = validation_generator, verbose = 1, validation_steps=3)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 543 steps, validate for 3 steps
Epoch 1/5
543/543 [==============================] - 517s 952ms/step - loss: 2.9198 - accuracy: 0.1624 - val_loss: 2.8580 - val_accuracy: 0.1376
Epoch 2/5
543/543 [==============================] - 538s 991ms/step - loss: 2.1496 - accuracy: 0.3439 - val_loss: 2.6460 - val_accuracy: 0.2090
Epoch 3/5
543/543 [==============================] - 504s 928ms/step - loss: 1.7603 - accuracy: 0.4477 - val_loss: 2.3300 - val_accuracy: 0.2751
Epoch 4/5
543/543 [==============================] - 510s 940ms/step - loss: 1.4799 - accuracy: 0.5293 - val_loss: 1.8793 - val_accuracy: 0.3757
Epoch 5/5
543/543 [==============================] - 498s 917ms/step - loss: 1.2780 - accuracy: 0.5873 - val_loss: 1.9630 - val_accuracy: 0.3836


In [30]:
model.save(current_dir+"\\model.h5")

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()